In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm

# MRL models

You will need to have model predictions and softmax values beforehand stored for this. 

In [7]:
N_seeds=30
import collections

In [8]:
ex = [];ttt=[]
for l in range(N_seeds):
    idx = np.random.choice(50000, 10000, replace=False) # True thresholding
    ex.append(idx)
    greedy_thrsh=[]
    gt = torch.load("ground_truth/gt_dataset=V1.pth")[idx].cpu()
    softmax=torch.load("softmax/softmax_sh=False_mh=True_ff=False_feat_dim=2048_dataset=V1.pth")[:, idx, :].cpu()
    confidence_, predictions_ = torch.max(softmax, dim=-1)
    n=len(gt)


    print(predictions_.shape)
    print(confidence_.shape)

    # confidence_ = confidence_.cuda()
    # predictions_ = predictions_.cuda()


    thrsh = np.linspace(0.1,1, 100)
    from matplotlib.pyplot import figure
#     figure(figsize=(20, 10), dpi=80)
    acc_all=[]
    for d1 in range(8):
        for d2 in range(d1+1, d1+2):
            print(2**(d1+3), 2**(d2+3))
            confidence_8, predictions_8 = confidence_[d1], predictions_[d1]
            confidence_128, predictions_128 = confidence_[d2], predictions_[d2]
            print(confidence_8.shape)
            softmax_ensemble_8_128 = (softmax[d1] + softmax[d2])/2
            ensemble_confidence, ensemble_predictions = torch.max(softmax_ensemble_8_128, dim=-1)
            print("Ensemble Accuracy",((ensemble_predictions==gt).sum())/n)
            acc=[]; FLOPS=[]; lower_=[]
            for t in thrsh:
                preds= torch.zeros(n); preds=preds.long()
                idx_8 = (confidence_8>t) #torch.nonzero((confidence_8>t).int()).squeeze()
                idx_128 =(confidence_8<=t) #torch.nonzero((confidence_8<t).int()).squeeze()
                n1, n2= (idx_8.sum()).item(), (idx_128.sum()).item()
    #             print(n1, n2)
                FLOPS.append((((2**(d1+3))/8)*n1+((2**(d2+3))/8)*n2)/(n1+n2))
                preds[idx_8] = predictions_8[(confidence_8>t)]
                preds[idx_128] = predictions_128[(confidence_8<=t)]
                acc.append(100*(((preds==gt).sum())/n).cpu().numpy())

            acc = np.asarray(acc)
            max_acc = -np.asarray(sorted(-acc))[0]
            best_thrhs = thrsh[np.nonzero(acc==max_acc)[0]].min()
            max_idx =(np.asarray(acc)).argmax()
            print("Max Cascade Performance",  acc[max_idx], best_thrhs)
            greedy_thrsh.append(best_thrhs)
            acc_all.append((np.asarray(FLOPS).min(), d1, d2))
    
    ttt.append(greedy_thrsh)
#     plt.show()

torch.Size([9, 10000])
torch.Size([9, 10000])
8 16
torch.Size([10000])
Ensemble Accuracy tensor(0.7367)
Max Cascade Performance 73.68999719619751 0.7
16 32
torch.Size([10000])
Ensemble Accuracy tensor(0.7504)
Max Cascade Performance 75.06999969482422 0.8181818181818181
32 64
torch.Size([10000])
Ensemble Accuracy tensor(0.7578)
Max Cascade Performance 76.02999806404114 0.7727272727272727
64 128
torch.Size([10000])
Ensemble Accuracy tensor(0.7616)
Max Cascade Performance 76.21999979019165 0.4818181818181818
128 256
torch.Size([10000])
Ensemble Accuracy tensor(0.7641)
Max Cascade Performance 76.46999955177307 0.509090909090909
256 512
torch.Size([10000])
Ensemble Accuracy tensor(0.7652)
Max Cascade Performance 76.66000127792358 0.6454545454545454
512 1024
torch.Size([10000])
Ensemble Accuracy tensor(0.7678)
Max Cascade Performance 76.80000066757202 0.5363636363636364
1024 2048
torch.Size([10000])
Ensemble Accuracy tensor(0.7682)
Max Cascade Performance 76.81999802589417 0.509090909090909


In [9]:
sel = []
for i in ex:
    u = np.zeros(50000)
    u[i]=1
    sel.append(np.nonzero(1-u)[0])

In [ ]:
temp3

In [ ]:
temp2=[[] for l in range(8)]
temp3=[[] for l in range(8)]
temp=[[] for l in range(8)]
for l in range(8):
    print(l+1)
    for foo in range(N_seeds):
        gt = torch.load("ground_truth/gt_dataset=V1.pth")[sel[foo]].cpu()
        softmax=torch.load("softmax/softmax_sh=False_mh=True_ff=False_feat_dim=2048_dataset=V1.pth")[:, sel[foo], :].cpu()
        confidence_, predictions_ = torch.max(softmax, dim=-1)
        greedy_thrsh = ttt[foo]

        dim_all, cascade_all=[], []
        for max_cascading in range(l+1, l+2):
            acc=0
            model=[]
            for i in tqdm(range(predictions_.shape[-1])):
                foo=True
                for j in range(max_cascading):
                    if confidence_[j, i] > greedy_thrsh[j]:
                        j_ = j; foo=False
                        break
                if foo:
                    j_=max_cascading
                model.append(j_)
                acc+=(predictions_[j_, i]==gt[i]).sum()   
            counter=collections.Counter(model)
            temp[l].append(counter)
            probs = {2**(j+3): counter[j]/len(gt) for j in counter.keys()}
            dim=0
            for k in probs.keys():
                print(k)
                dim+= k*probs[k]

#             print(f"Expected Dimensionality with Greedy Cascade {dim} and corresponding performance {(acc/len(gt)).item()}")
            dim_all.append(dim)
            cascade_all.append((acc/len(gt)).item())
        temp2[l].append(dim_all)
        temp3[l].append(cascade_all)

In [11]:
temp2 = np.asarray(temp2)
temp3 = np.asarray(temp3)

In [12]:
temp2= temp2.squeeze()

In [13]:
temp3= temp3.squeeze()

### EXPECTED DIMENSIONS AND STD

In [14]:
print(temp2.mean(axis=-1))
print(temp2.std(axis=-1))

[ 13.18502     18.02794     25.73146     35.28671333  45.66122
  62.05503333  87.92042    121.67231333]
[ 0.72321347  1.60812216  2.80969956  5.28050878  7.60811513 10.94328704
 20.18233146 32.94076334]


### CORRESPONDING PERFORMANCE

In [15]:
print(temp3.mean(axis=-1))
print(temp3.std(axis=-1))

[0.73790917 0.75242666 0.76048249 0.76273834 0.76418666 0.76514667
 0.76533666 0.76541667]
[0.00103216 0.00091551 0.00139763 0.00170652 0.00188398 0.0019996
 0.002036   0.0020418 ]


In [16]:
ttt = np.asarray(ttt) # threshold for different random seeds

### CORRESPONDING THRESHOLDS

In [19]:
print(ttt.mean(axis=0))
print(ttt.std(axis=0))

[0.78636364 0.70939394 0.7930303  0.65151515 0.55333333 0.55636364
 0.52212121 0.44484848]
[0.07305796 0.11164888 0.07465931 0.1491451  0.05973926 0.06952812
 0.13982837 0.11729113]
